<a href="https://colab.research.google.com/github/Tanvir-Anzum/React-project/blob/main/DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New section

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#importing necissary libraries

from numpy.random import seed
seed(101)

import pandas as pd
import numpy as np

import tensorflow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import os
from os import listdir
import cv2

import imageio
import skimage
import skimage.io
import skimage.transform

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from skimage.io import imread
import itertools
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
training_data = '/content/drive/MyDrive/Deep Learning/BreaKHis 400X/teain'

test_data = '/content/drive/MyDrive/Deep Learning/BreaKHis 400X/test'


In [ ]:
# Pytorch tlements
import torch
import torchvision
import torchvision.models as models
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, ConcatDataset
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import random

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime


In [ ]:

gpu_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(gpu_device)

cpu


In [ ]:
import glob
import pandas as pd

# set the directory path
dir_path = "/content/drive/MyDrive/Deep Learning/BreaKHis 400X"




In [ ]:
import torch

from torchvision import transforms
import os

transform = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,)),
     transforms.Resize((246, 246)),
])
augmented_datasets = [ ]

# Create datasets for training. and validation
training_set = torchvision.datasets.ImageFolder('/content/drive/MyDrive/Deep Learning/BreaKHis 400X/train', transform=transform)
test_set = torchvision.datasets.ImageFolder('/content/drive/MyDrive/Deep Learning/BreaKHis 400X/test', transform=transform)

original_train_set = torchvision.datasets.ImageFolder('/content/drive/MyDrive/Deep Learning/BreaKHis 400X/train', transform=transform)


# Define the number of times to augment the dataset
num_augmentations = 10

# Define the transforms for data augmentation
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(246),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.Resize(246)  # Add the missing resize transformation
])

for _ in range(num_augmentations):
     # Apply the data augmentation to the original dataset
     augmented_train_set = torchvision.datasets.ImageFolder('/content/drive/MyDrive/Deep Learning/BreaKHis 400X/train', transform=train_transform)

     # Add the augmented dataset to the li
     augmented_datasets.append(augmented_train_set)

# Concatenate the augmented datasets
augmented_train_set = ConcatDataset([original_train_set] + augmented_datasets)

# Print the size of the augmented dataset
print("Size of augmented dataset:", len(augmented_train_set))

Size of augmented dataset: 12639


In [ ]:
# import torchvision.transforms as transforms

# # Load the image tensor from the dataset
# img_tensor, label = training_set[1]

# directory = '/content/drive/MyDrive/My Graphs/'
# path = '/content/drive/MyDrive/My Graphs/ka.png'

# if not os.path.exists(path):
#     os.makedirs(path)
# if not os.path.exists(directory):
#     os.makedirs(directory)

# # Define the data augmentation transformations
# transform = transforms.Compose([
#     transforms.RandomResizedCrop(224),
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomRotation(10),
#     transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
# ])

# # Apply the transformations to the image tensor
# augmented_img_tensor = transform(img_tensor)

# # Convert the tensor back to a PIL image for visualization
# augmented_img = transforms.ToPILImage()(augmented_img_tensor)

# # Show the original and augmented images side-by-side
# import matplotlib.pyplot as plt
# fig, axs = plt.subplots(1, 2, figsize=(10,5))
# axs[0].imshow(img_tensor.permute(1, 2, 0))
# axs[0].set_title('Original')
# axs[1].imshow(augmented_img)
# axs[1].set_title('Augmented')
# plt.savefig(path + 'ka.png')
# plt.show()


In [ ]:
batch_size_train=128
batch_size_val=32
batch_size_test=32

In [ ]:
from torch.utils.data import random_split

train_size = int(len(augmented_train_set) * 0.8)
val_size = len(augmented_train_set) - train_size

# Split training set into training and validation sets
train_set, val_set = random_split(augmented_train_set, [train_size, val_size])

# Create data loaders for training, validation, and test sets
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size_train, shuffle=False)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size_val, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size_test, shuffle=False)


In [ ]:
classes = ('benign', 'malignant')

# Report split sizes
print('Training set has {} instances'.format(len(train_set)))
print('Test set has {} instances'.format(len(test_set)))

Training set has 10111 instances
Test set has 545 instances


In [ ]:
# Creating initial batch
loss_fn = torch.nn.CrossEntropyLoss()

dummy_outputs = torch.rand(batch_size_train, 2)
dummy_labels = torch.randint(low=0, high=2, size=(batch_size_train,))

print(dummy_outputs)
print(dummy_labels)

loss = loss_fn(dummy_outputs, dummy_labels)
print('Total loss for this batch: {:.4f}'.format(loss.item()))

tensor([[4.7848e-02, 5.1845e-01],
        [1.7851e-01, 6.1248e-01],
        [8.7864e-01, 9.9556e-01],
        [5.0684e-01, 9.4200e-01],
        [6.9540e-02, 7.1553e-01],
        [4.5648e-02, 1.7867e-01],
        [5.7181e-01, 6.4342e-01],
        [5.4594e-01, 5.0837e-02],
        [1.9421e-01, 4.0214e-01],
        [4.4397e-01, 9.0029e-01],
        [2.5563e-01, 4.2380e-01],
        [9.5598e-01, 5.5512e-01],
        [8.3341e-02, 9.4308e-01],
        [9.5607e-01, 9.0120e-01],
        [7.6172e-03, 9.5079e-01],
        [1.0341e-02, 9.9554e-01],
        [5.4192e-01, 4.9333e-01],
        [8.6872e-01, 7.3926e-01],
        [9.7807e-01, 7.3217e-01],
        [7.6662e-01, 8.0082e-02],
        [4.3440e-01, 9.0700e-01],
        [6.2786e-01, 5.8752e-01],
        [8.3885e-01, 6.2316e-01],
        [6.2161e-01, 3.0022e-01],
        [5.3875e-01, 5.2412e-01],
        [8.3599e-01, 7.3160e-01],
        [3.5734e-01, 1.9525e-01],
        [3.0747e-01, 8.3986e-01],
        [7.8130e-01, 7.6854e-01],
        [5.952

In [ ]:
# # Image display function
# def matplotlib_imshow(img, one_channel=False):
#     if one_channel:
#         img = img.mean(dim=0)
#     img = img / 2 + 0.5     # unnormalize
#     print(img)
#     npimg = img.numpy()
#     if one_channel:
#         plt.imshow(npimg, cmap='Greys')
#     else:
#         plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [ ]:
# from sklearn.decomposition import PCA

# # Get the features from the dataset
# features = None
# for images, labels in original_train_set:
#     images = images.numpy()
#     images = images.reshape(images.shape[0], -1)
#     images -= np.mean(images, axis=0)
#     features = images if features is None else np.vstack((features, images))

# # Define the number of components for PCA
# n_components = 3

# # Perform PCA on the features
# pca = PCA(n_components=n_components)
# pca_result = pca.fit_transform(features)


# pca_result = pca.fit_transform(features)

# # Perform PCA on the features
# pca = PCA(n_components=n_components)
# pca_result = pca.fit_transform(features)


# # Print the explained variance ratio of each PCA component
# print("Explained variance ratio:", pca.explained_variance_ratio_)


In [ ]:
# import seaborn as sns

# # Get the features from the dataset
# features = None
# num_images = 0;
# labels = []
# for images, label in original_train_set:
#     images = images.numpy()
#     images = images.reshape(images.shape[0], -1)
#     features_batch = pca.transform(images)
#     features = features_batch if features is None else np.concatenate((features, features_batch), axis=0)
#     labels.append(label)
#     num_images += images.shape[0]

# # Create a DataFrame from the PCA results and the labels
# df = pd.DataFrame(data=features, columns=[f"pca-{i+1}" for i in range(n_components)])
# df["label"] = labels[:num_images]

# # Plot the first two dimensions of the PCA results
# plt.figure(figsize=(16,10))
# g_pca_2d = sns.scatterplot(
#     x="pca-1", y="pca-2",
#     hue="label",
#     palette = sns.color_palette("tab10", n_classes),
#     data=df,
#     legend="full",
#     alpha=1
# )
# g_pca_2d.legend(loc='upper right')
# g_pca_2d.set_title("First two dimensions of PCA-compressed-data")


In [ ]:

# # Extract features from the images using a pre-trained CNN
# # This example uses the ResNet50 architecture
# cnn = models.resnet50(pretrained=True)
# cnn.eval()
# features = []
# labels = []
# for img, label in original_train_set:
#     with torch.no_grad():
#         feature = cnn.forward(img.unsqueeze(0)).squeeze().numpy()
#     features.append(feature)
#     labels.append(label)
# features = np.array(features)
# labels = np.array(labels)

# # plt.scatter(pca_result[:, 0], pca_result[:, 1],
# #             c=digits.target, edgecolor='none', alpha=0.5,
# #             cmap=plt.cm.get_cmap('spectral', 10))
# # plt.xlabel('component 1')
# # plt.ylabel('component 2')
# plt.colorbar();

In [ ]:
# # Create a scatter plot of the first three principal components
# num_classes = 3
# colors = ['green' if label == 0 else 'orange' for label in labels]

# plt.scatter(pca_result[:, 0], pca_result[:, 1], c=pca_result[:, 2],
#             edgecolor='none', alpha=0.5, cmap='spring', s=10)
# # Set the path to the folder where you want to save the file
# path = '/content/drive/MyDrive/My Graphs/'

# if not os.path.exists(directory):
#     os.makedirs(directory)


# plt.xlabel('component 1')
# plt.ylabel('component 2')
# plt.ylabel('component 3')
# plt.colorbar()


# # Save the file with a specific name
# plt.savefig(path + 'output.png')
# plt.show()

In [ ]:
# dataiter = iter(train_loader)
# images, labels = next(dataiter)

# img_grid = torchvision.utils.make_grid(images)
# print(img_grid.shape)
# matplotlib_imshow(img_grid, one_channel=False)

# print(images.shape)
# print('  '.join(classes[labels[j]] for j in range(batch_size_train)))

In [ ]:
# class my_vgg16(nn.Module):
#     def __init__(self, originalModel, num_of_channels=3, num_classes=2):
#         super(my_vgg16, self).__init__()
#         self.features = nn.Sequential(*list(originalModel.features)[:])

#         self.fully_connected_layers = nn.Sequential(
#             nn.Linear(512 * 7 * 7, 4096),
#             nn.ReLU(),
#             nn.Dropout(p=0.5),
#             nn.Linear(4096, 4096),
#             nn.ReLU(),
#             nn.Dropout(p=0.5),
#             nn.Linear(4096, num_classes),
#         )

#     def forward(self, x):
#         x = self.features(x)
#         x = x.reshape(x.shape[0], -1)
#         x = self.fully_connected_layers(x)
#         return x

In [ ]:
# # Build model and check its properties
# model_vgg16 = torch.hub.load('pytorch/vision:v0.8.0', 'vgg16', pretrained=True)
# print(model_vgg16)
# for i, param in enumerate(model_vgg16.parameters()):
#     param.requires_grad = False
#     print(i)
#     print(param.requires_grad)
# model = my_vgg16(model_vgg16)
# print(model)
# for i, param in enumerate(model.parameters()):
#     print(i)
#     print(param.requires_grad)

# # If GPU is available
# if gpu_device == 'cuda':
#     model.to(gpu_device)           ### GPU

In [ ]:
 class my_resnet50(nn.Module):
    def __init__(self, originalModel, num_of_channels=3, num_classes=2):
        super(my_resnet50, self).__init__()
        self.features = nn.Sequential(*list(originalModel.children())[:-1])
        self.fully_connected_layers = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.fully_connected_layers(x)
        return x

In [ ]:

# Load ResNet50 model
model_resnet50 = torch.hub.load('pytorch/vision:v0.8.0', 'resnet50', pretrained=True)

# Freeze all the parameters in the ResNet50 model
for param in model_resnet50.parameters():
    param.requires_grad = False

# Create custom model by replacing the last layer with our fully connected layers
model = my_resnet50(model_resnet50)

# Set the gradients for the fully connected layers to be trainable
for param in model.fully_connected_layers.parameters():
    param.requires_grad = True

# If GPU is available
# if gpu_device == 'cuda':
    # model.to(gpu_device)


num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {num_params}")


num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {num_params}")

Downloading: "https://github.com/pytorch/vision/zipball/v0.8.0" to /root/.cache/torch/hub/v0.8.0.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 227MB/s]


Number of trainable parameters: 2624002
Number of trainable parameters: 2624002


In [ ]:
class my_resnet101(nn.Module):
    def __init__(self, originalModel, num_of_channels=3, num_classes=2):
        super(my_resnet101, self).__init__()
        self.features = nn.Sequential(*list(originalModel.children())[:-1])
        self.fully_connected_layers = nn.Sequential(
            nn.Linear(2048, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(1024, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.fully_connected_layers(x)
        return x

# Load ResNet101 model
model_resnet101 = torch.hub.load('pytorch/vision:v0.8.0', 'resnet101', pretrained=True)

# Freeze all the parameters in the ResNet101 model
for param in model_resnet101.parameters():
    param.requires_grad = False

# Create custom model by replacing the last layer with our fully connected layers
model = my_resnet101(model_resnet101)


num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {num_params}")

# Set the gradients for the fully connected layers to be trainable
for param in model.fully_connected_layers.parameters():
    param.requires_grad = True

# If GPU is available
# if gpu_device == 'cuda':
    # model.to(gpu_device)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.8.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:02<00:00, 82.0MB/s]


Number of trainable parameters: 12590082


In [ ]:
import torch.nn as nn
import torch.optim.lr_scheduler as lr_scheduler

# # Optimizers initiation

optimizer = torch.optim.SGD(model.parameters(), lr= 0.01, momentum=0.9)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.8)

# optimizer = torch.optim.Adam(model.parameters())


# initialize lists to store train and validation accuracy
train_acc_list = []
val_acc_list = []

In [ ]:
# One loop training function
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.
    running_corrects = 0
    total_samples = 0


    for i, data in enumerate(train_loader):

        # Every data instance is an input + label pair
        inputs, labels = data

        # If GPU is available
        if gpu_device == 'cuda':
            inputs, labels = inputs.cuda(), labels.cuda()           ### GPU

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        running_corrects += torch.sum(preds == labels.data)
        total_samples += inputs.size(0)

        if i % 12 == 1:
            last_loss = running_loss / batch_size_train
            accuracy = running_corrects.double() / total_samples
            print("\rbatch {}/{}, loss: {:.4f}, accuracy: {:.2%}".format(i + 1, len(train_loader), last_loss, accuracy), end='', flush=True)
            print("\rbatch {}/{}, loss: {:.4f}".format(i+1, len(train_loader), last_loss, end='', flush=True))
            tb_x = epoch_index * len(train_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.
            running_corrects = 0
            total_samples = 0

    return last_loss

In [ ]:
pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [ ]:
# Initializing TensorBoard data writer
import datetime
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
from tensorboardX import SummaryWriter
writer = SummaryWriter('runs/Breast_cancer_trainer_{}'.format(timestamp))
epoch_number = 0

model_dir = '/content/drive/MyDrive/models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

model_path = os.path.join(model_dir, 'model_{}_{}'.format(timestamp, epoch_number))

In [ ]:
total_epochs = 2
best_vloss = 1000000.
start_epoch = 0
epoch_number = 1

# Before the loop
all_vlabels = []
all_vpreds = []
all_voutputs = []

from sklearn.metrics import accuracy_score, f1_score, precision_score, roc_auc_score

f1_score_list = []
precision_list = []
auc_list = []

for param_group in optimizer.param_groups:
    param_group['lr'] = 0.001
    print(param_group['lr'])

# check if the model_path file exists, and load the weights if it does

print("Optimizer:", type(optimizer).__name__)

num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {num_params}")


for epoch in range(total_epochs):
    print('EPOCH {}:'.format(epoch_number + 1))


    # Find the latest saved model in the model directory
    model_dir = '/content/drive/MyDrive/models/'
    model_files = os.listdir(model_dir)

    model_files.sort(key=lambda x: os.path.getctime(os.path.join(model_dir, x)))
    last_saved_model = model_files[-1]
    model_path = os.path.join(model_dir, last_saved_model)
    model_path = '/content/drive/MyDrive/models/model_20230624_050006_1'


    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    # optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    for param_group in optimizer.param_groups:
        param_group['lr'] = 0.002

    for param_group in optimizer.param_groups:
        print(param_group['lr'])
        print('Loaded pre-trained model:', model_path)

    # Turn gradient tracking on, and train
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)

    # Turn gradient tracking off
    model.train(False)

    # Run validation
    running_vloss = 0.0
    running_vcorrects = 0.
    total_vpreds = 0

    for i, vdata in enumerate(val_loader):
        vinputs, vlabels = vdata
          # if gpu_device == 'cuda':1
          # vinputs, vlabels = vinputs.cuda(), vlabels.cuda()

        voutputs = model(vinputs)
        vloss = loss_fn(voutputs, vlabels)
        vpreds = torch.argmax(voutputs, dim=1)
        running_vloss += vloss.item()
        running_vcorrects += torch.sum(vpreds == vlabels.data)
        total_vpreds += len(vlabels)
        all_vlabels.append(vlabels)
        all_vpreds.append(vpreds)
        all_voutputs.append(voutputs)

        vlabels_all = torch.cat(all_vlabels, dim=0).flatten().detach().numpy()
        vpreds_all = torch.cat(all_vpreds, dim=0).flatten().detach().numpy()
        concatenated_outputs = torch.cat(all_voutputs, dim=0)
        flattened_outputs = concatenated_outputs.view(-1).detach().numpy()



        print("vlabels shape:", vlabels_all.shape)
        print("vpreds shape:", vpreds_all.shape)
        print("outputs shape:", flattened_outputs.shape)


    avg_vloss = running_vloss / (i + 1)
    avg_vacc = running_vcorrects / total_vpreds
    print('LOSS valid {:.4f}, ACC valid {:.4f}'.format(avg_vloss, avg_vacc))






    model_path = os.path.join(model_dir, 'model_{}_{}'.format(timestamp, epoch_number))
    torch.save({
        'epoch': epoch_number,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_vloss': best_vloss,
        'val_acc_list': val_acc_list
    }, model_path)

    f1 = f1_score(vlabels_all, vpreds_all, average='weighted')  # Use scikit-learn's f1_score
    precision = precision_score(vlabels_all, vpreds_all, average='weighted')  # Use scikit-learn's precision_score

    print('f1 : ',f1)

    # Append the train and validation accuracy to the lists

    val_acc_list.append(avg_vacc)
    f1_score_list.append(f1)
    precision_list.append(precision)

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss

    # Update the learning rate
    # lr_scheduler.step()


    epoch_number += 1

0.001
Optimizer: SGD
Number of trainable parameters: 12590082
EPOCH 2:
0.002
Loaded pre-trained model: /content/drive/MyDrive/models/model_20230624_050006_1


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


batch 2/79, loss: 0.0034
batch 14/79, loss: 0.0185
batch 26/79, loss: 0.0154
batch 38/79, loss: 0.0150


In [ ]:
import torch
# model_path = '/content/drive/MyDrive/models/model_20230319_151031_13'
6g# checkpoint = torch.load(model_path)

In [ ]:
# # plot the train and validation accuracy for each epoch
# plt.plot(train_acc_list, label='Train accuracy', color='blue', linestyle='solid')
# plt.plot(val_acc_list, label='Validation accuracy', color='orange', linestyle='dashed')
# # add a title,6 labels, and legend
# plt.title('Accuracy over Epochs')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.legend()
import matplotlib.pyplot as plt
import os

# train_acc_list = checkpoint.get('train_acc_list', [])
# val_acc_list = checkpoint.get('val_acc_list', [])


# create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# create a figure with a single subplot
fig, ax = plt.subplots(figsize=(10, 5))

# Set the path to the folder where you want to save the file
directory = '/content/drive/MyDrive/My Graphs/'
path = '/content/drive/MyDrive/My Graphs/neww.png'

if not os.path.exists(path):
    os.makedirs(path)
if not os.path.exists(directory):
    os.makedirs(directory)

print(f1_score_list)

# plot the train and validation accuracy for each epoch as a line plot
ax1.plot(train_acc_list, label='Train accuracy')
ax1.plot(val_acc_list, label='Validation accuracy')
# set the y-axis range manually
ax1.set_ylim([0.85, 0.96])
ax1.set_title('Accuracy over Epochs')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()

# plot the train and validation accuracy for each epoch as a scatter plot
ax2.scatter(range(1, len(train_acc_list)+1), train_acc_list, label='Train accuracy', color='blue')
ax2.scatter(range(1, len(val_acc_list)+1), val_acc_list, label='Validation accuracy', color='orange')
ax2.set_title('Accuracy over Epochs')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.legend()

# adjust the layout and spacing between subplots
fig.tight_layout()
# Save the file with a specific name
plt.savefig(path + 'neww.png')
plt.show()

In [ ]:

  # Assuming you have a test_loader that contains the test data
  model_path = '/content/drive/MyDrive/models/model_20230624_050006_1'
  checkpoint = torch.load(model_path)
  model.load_state_dict(checkpoint['model_state_dict'])
  total_tpreds = 0.0
  running_tcorrects = 0.0

  # Turn gradient tracking off
  model.train(False)

  for i, tdata in enumerate(test_loader):
        tinputs, tlabels = tdata
        # If GPU is available
         # if gpu_device == 'cuda':
              ### GPU
        toutputs = model(tinputs)
        # vloss = loss_fn(voutputs, vlabels)
        tpreds = torch.argmax(toutputs, dim=1)
        # running_vloss += vloss.item()
        running_tcorrects += torch.sum(tpreds == tlabels.data)
        total_tpreds += len(tlabels)


    # avg_vloss = running_vloss / (i + 1)
  avg_tacc = running_tcorrects / total_tpreds
  print('ACC {:.4f}'.format(avg_tacc))

In [ ]:
epoch_number